In [1]:
import yt
import matplotlib
import matplotlib.pyplot as plt

import os
import shutil
import numpy as np

In [2]:
yt.funcs.mylog.setLevel(50)

FVS_path = '/home/maikel/Development/FiniteVolumeSolver'
extra_path = '{}/extra'.format(FVS_path)

level_integrator_path = '{}/build_2D-Debug/TravellingVortex_old/Debug'.format(FVS_path)
solver_path = '{}/build_2D-Debug/TravellingVortex/Debug'.format(FVS_path)

#level_integrator_path = '{}/build_2D-Debug/TravellingVortex_old/Plotfiles'.format(FVS_path)
#solver_path = '{}/build_2D-Debug/TravellingVortex/Plotfiles'.format(FVS_path)


def LoadFVS(base_dir, step_dir, partname, timestep):
  path = os.path.join(base_dir, step_dir, partname+'plt{:09d}'.format(timestep))
  shutil.copy2('{}/yt/WarpXHeader'.format(extra_path), path)
  shutil.copy2('{}/yt/warpx_job_info'.format(extra_path), path)
  return yt.load(path)

substeps = [
    'BK19_pre-step',
    'BK19_advect',
    'BK19_advect-backward',
    'BK19_advect-backward-forward',
    'BK19_advect-backward-forward-advect',
    'BK19_advect-backward-forward-advect-backward'
]


In [4]:
def CompareField(dataset1, dataset2, field, abs_tolerance=1e-8):
    if (dataset1.domain_dimensions != dataset2.domain_dimensions).all():
        return False
    dims = dataset1.domain_dimensions

    p1 = yt.SlicePlot(dataset1, 'z', field)
    p1.set_buff_size([dims[0], dims[1]])
    field1 = np.array(p1.frb[field])
    
    p2 = yt.SlicePlot(dataset2, 'z', field)
    p2.set_buff_size([dims[0], dims[1]])
    field2 = np.array(p2.frb[field])

    dField = field2 - field1
    d_norm = np.linalg.norm(dField, np.inf)
    if d_norm > abs_tolerance:
        print("{}:".format(field))
        figs, axs = plt.subplots(1, 3)
        axs[0].imshow(field1, origin='lower')
        axs[1].imshow(field2, origin='lower')
        axs[2].imshow(dField, origin='lower')
        plt.show()
        print("Error norm for field variable '{}' is too large: {}".format(field, d_norm))
        print(dField)
        return False
    return True

def CompareMultipleFields(ds1, ds2, fields):
    comparisons = [CompareField(ds1, ds2, field) for field in fields]
    return all(comparisons)
    

fields = {
    '': ["Density", "Momentum_0", "Momentum_1", "PTdensity", "PTinverse"],
    'BK19_pre-step': ["Density", "Momentum_0", "Momentum_1", "PTdensity", "PTinverse", "pi_nd2cellavg"],
    'BK19_advect': ["Density", "Momentum_0", "Momentum_1", "PTdensity", "PTinverse", "pi_nd2cellavg"],
    'BK19_advect-backward': ["Density", "Momentum_0", "Momentum_1", "PTdensity", "PTinverse", "Momentum_corr0", "Momentum_corr1", "alpha_nd2cellavg", "pi_nd2cellavg", "solution_nd2cellavg", "sigma"],
    'BK19_advect-backward-forward': ["Density", "Momentum_0", "Momentum_1", "PTdensity", "PTinverse", "div_Pv_nd2cellavg", "pi_nd2cellavg", "Momentum_corr0", "Momentum_corr1", "Pi_correction_nd2cellavg", "Pu", "Pv"],
    'BK19_advect-backward-forward-advect': ["Density", "Momentum_0", "Momentum_1", "PTdensity", "PTinverse", "pi_nd2cellavg", "Pu", "Pv"],
    'BK19_advect-backward-forward-advect-backward': ["Density", "Momentum_0", "Momentum_1", "PTdensity", "PTinverse", "Momentum_corr0", "Momentum_corr1", "alpha_nd2cellavg", "pi_nd2cellavg", "solution_nd2cellavg", "sigma"],
}

def CompareSubstep(substep, timestep, partition='partition_0_'):
    ds1 = LoadFVS(level_integrator_path, substep, 'partition_0_', timestep)
    ds2 = LoadFVS(solver_path, substep, 'partition_0_', timestep)
    return CompareMultipleFields(ds1, ds2, fields[substep])

In [5]:
for substep in substeps:
    print("Comparing substep {}".format(substep))
    if not CompareSubstep(substep, 1):
        print("Failed to compare substep '{}'.".format(substep))
        break